### SQLITE3

In [1]:
import sqlite3

db_name = './database/my_books.db'

### 데이터 등록

In [2]:
import sqlite3

#데이터 입력 함수
def insert_books(db_name):
    """
    데이터베이스 테이블에 데이터를 등록하는 함수
    Args:
        db_name : Database Name
    Returns : 
        is_success : Boolean 
    """
    is_success = True
    
    try:
        #데이터베이스 커넥션 생성
        conn = sqlite3.connect(db_name)
        #커서확보
        cur = conn.cursor()
        #데이터 입력 SQL1
        db_sql = "INSERT INTO my_books VALUES ('메가트랜드', '2002.03.02', 'A', 200,0)"
        cur.execute(db_sql)
        
        #데이터 입력 sql2
        db_sql = 'INSERT INTO my_books VALUES (?, ?, ?, ?, ?)'
        cur.execute(db_sql, ('인더스트리 4.0', '2016.07.09','B', 584, 1))
        
        #데이터 입력 sql3
        books = [
            ('유니콘 스타트업', '2011.07.15','A', 248, 1),
            ('빅데이터 마케팅', '2012.08.25','A', 296, 1),
            ('사물인터넷 전망', '2013.08.22','B', 526, 0)
        ]
        cur.executemany(db_sql, books)     
        
    except:
        is_success = False
        print("Database Error!")
    
    finally:
        if is_success:
            #데이터베이스 반영
            conn.commit()
        else:
            #데이터베이스 철회
            conn.rollback()
        conn.close()
        
    return is_success
            

In [3]:
if insert_books(db_name):
    print('데이터가 성공적으로 등록되었습니다.')
else :
    print('데이터가 등록되지 않았습니다')

데이터가 성공적으로 등록되었습니다.


- 데이터 조회, SELECT

### SQLITE3

In [4]:
import sqlite3

db_name = './database/my_books.db'

### 데이터조회

In [5]:
import pandas as pd

def getBooksDF(books):
    ret_df = pd.DataFrame()
    
    title = list()
    published_date = list()
    publisher = list()
    pages = list()
    recommendation = list()
    
    column_name = ['title', 'published_date', 'publisher', 'pages', 'recommendation']
    
    for book in books:
        print(book)
        # for value in book:
        #     print(value, end=" | ")
        title.append(book[0])
        published_date.append(book[1])
        publisher.append(book[2])
        pages.append(book[3])
        recommendation.append(book[4])
        
    data = {
        'title'          : title         ,
        'published_date' : published_date,
        'publisher'      : publisher     ,
        'pages'          : pages         ,
        'recommendation' : recommendation
    }        
    
    
    ret_df = pd.DataFrame(data, columns = column_name)
    
    return ret_df
        

In [6]:
import sqlite3
import pandas as pd

def select_all_books(db_name):
    """
    전체 데이터를 조회하는 함수
    Args:
        db_name : Database Name
    Returns :
        is_success : Boolean 
        ret_df : DataFrame of books
    """
    ret_df = pd.DataFrame()
    is_success = True
    
    try:
        # 데이터베이스 커넥션 생성
        conn = sqlite3.connect(db_name)
        # 커서 확보
        cur = conn.cursor()
        # 조회용 SQL 실행
        db_sql = 'SELECT * FROM my_books'
        cur.execute(db_sql)
        # 조회한 데이터 불러오기
        print('[1] 전체 데이터 출력하기')
        books = cur.fetchall()
        
        ret_df = getBooksDF(books)
        
        #데이터출력하기
        #for book in books:
        # print(book)
        
    except:
        is_success = False
        print("Database Error!")
        
    finally:
        #데이터베이스 커넥션 닫기
        conn.close()
    
    return is_success, ret_df
        

In [7]:
is_success, books_df = select_all_books(db_name)
if is_success:
    print('조회된 데이터는 총 %d 건 입니다.'%len(books_df))
else:
    print('데이터를 조회하지 못했습니다.')
    
books_df

[1] 전체 데이터 출력하기
('메가트랜드', '2002.03.02', 'A', 200, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
('메가트랜드', '2002.03.02', 'A', 200, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
조회된 데이터는 총 10 건 입니다.


,title,published_date,publisher,pages,recommendation
0,메가트랜드,2002.03.02,A,200,0
1,인더스트리 4.0,2016.07.09,B,584,1
2,유니콘 스타트업,2011.07.15,A,248,1
3,빅데이터 마케팅,2012.08.25,A,296,1
4,사물인터넷 전망,2013.08.22,B,526,0
5,메가트랜드,2002.03.02,A,200,0
6,인더스트리 4.0,2016.07.09,B,584,1
7,유니콘 스타트업,2011.07.15,A,248,1
8,빅데이터 마케팅,2012.08.25,A,296,1
9,사물인터넷 전망,2013.08.22,B,526,0


In [8]:
# 일부 조회용 함수
def select_some_books(db_name, number):
    """
    일부 데이터를 조회하는 함수
    Args:
        db_name : Database Name
        number  : Count of data to query
    Returns : 
        is_success : Boolean 
        ret_df : DataFrame of books
    """
    ret_df = pd.DataFrame()
    is_success = True
    
    try:
        conn = sqlite3.connect(db_name)
        cur=conn.cursor()
        db_sql = "SELECT * FROM my_books"
        cur.execute(db_sql)
        print("[2] 데이터 일부 출력하기")
        books = cur.fetchmany(number)
        
        ret_df = getBooksDF(books)
        
    except:
        is_success = False
        print('Database Error!!')
        
    finally:
        conn.close()
        
    return is_success, ret_df

In [9]:
#select_some_books[db_name, number =3]

is_success, books_df = select_some_books(db_name, number=3)
if is_success:
    print('조회된 데이터는 총 %d 건 입니다.'%len(books_df))
else:
    print('데이터를 조회하지 못했습니다.')
    
books_df

[2] 데이터 일부 출력하기
('메가트랜드', '2002.03.02', 'A', 200, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
조회된 데이터는 총 3 건 입니다.


,title,published_date,publisher,pages,recommendation
0,메가트랜드,2002.03.02,A,200,0
1,인더스트리 4.0,2016.07.09,B,584,1
2,유니콘 스타트업,2011.07.15,A,248,1


In [10]:
# 1개 조회용 함수
def select_one_book(db_name):
    """
    최상단 하나의 데이터를 조회하는 함수
    Args:
        db_name : Database Name
    Returns : 
        is_success : Boolean 
        ret_df : DataFrame of books
    """
    ret_df = pd.DataFrame()
    is_success = True
    
    try:
        conn = sqlite3.connect(db_name)
        cur = conn.cursor()
        db_sql = "SELECT * FROM my_books"
        cur.execute(db_sql)
        print('[3] 1개 데이터 출력하기')
        book = cur.fetchone()
        books = [book]
        ret_df = getBooksDF(books)
        
    except:
        is_success = False
        print('Database Error!')
        
    finally:
        conn.close()
        
    return is_success, ret_df

In [11]:
#select one book (db_name)

is_success, books_df = select_one_book(db_name)
if is_success:
    print('하나의 데이터를 성공적으로 조회하였습니다.')
else:
    print('데이터를 조회하지 못했습니다')

books_df

[3] 1개 데이터 출력하기
('메가트랜드', '2002.03.02', 'A', 200, 0)
하나의 데이터를 성공적으로 조회하였습니다.


,title,published_date,publisher,pages,recommendation
0,메가트랜드,2002.03.02,A,200,0


In [12]:
# 쪽수 많은 책 조회용 함수
def find_big_books(db_name):
    """
    조건에 맞는 데이터를 조회하는 함수
    조건 : 페이지수가 300쪽보다 큰 데이터
    Args:
        db_name : Database Name
    Returns : 
        is_success : Boolean 
        ret_df : DataFrame of books
    """
    ret_df = pd.DataFrame()
    is_success = True
    
    try:
        # 데이터베이스 커넥션 생성
        conn = sqlite3.connect(db_name) 

        # 커서 확보
        cur = conn.cursor()  

        # 조회용 SQL 실행
        # db_sql = "SELECT title, pages FROM my_books "
        db_sql = "SELECT * FROM my_books "
        db_sql+= "WHERE pages > 300"
        cur.execute(db_sql) 

        # 조회한 데이터 불러오기
        print('[4] 페이지 많은 책 출력하기')
        books = cur.fetchall()
        
        ret_df = getBooksDF(books)

    except:
        is_success = False
        print("Database Error!")
        
    finally : 
        # 데이터베이스 커넥션 닫기
        conn.close()
        
    return is_success, ret_df     

In [13]:
# find_big_books(db_name)

is_success, books_df = find_big_books(db_name)
if is_success:
    print('조건에 맞는 데이터는 총 %d 건 입니다.(조건:pages>300)'%len(books_df))

else:
    print('데이터를 조회하지 못했습니다.')
    
books_df

[4] 페이지 많은 책 출력하기
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
조건에 맞는 데이터는 총 4 건 입니다.(조건:pages>300)


,title,published_date,publisher,pages,recommendation
0,인더스트리 4.0,2016.07.09,B,584,1
1,사물인터넷 전망,2013.08.22,B,526,0
2,인더스트리 4.0,2016.07.09,B,584,1
3,사물인터넷 전망,2013.08.22,B,526,0


In [22]:
#find books by title

def find_books_by_title(db_name, title):
    """
    조건에 맞는 데이터를 조회하는 함수
    조건 : 제목 책으로 찾기
    Args:
        db_name : Database Name
    Returns : 
        is_success : Boolean 
        ret_df : DataFrame of books
    """
    ret_df = pd.DataFrame()
    is_success = True
    
    try:
        conn= sqlite3.connect(db_name)
        cur = conn.cursor()
        db_sql="SELECT * FROM my_books "
        db_sql+= "WHERE title LIKE '{}%' "
        #'LIKE {}'%는 한꺼번에-
        
        #print(db_sql.format(title))
        cur.execute(db_sql.format(title))
        
        print("[5] 책 제목으로 출력하기")
        books = cur.fetchall()
        
        ret_df = getBooksDF(books)
        
    except:
        is_success = False
        print('Database Error!')
        
    finally:
        conn.close()
        
    return is_success, ret_df

In [23]:
#find_books_by_title(db_name)

title = '빅데이터'

is_success, books_df = find_books_by_title(db_name, title)

if is_success:
    print('조건에 맞는 데이터는 총 %d 건 입니다' %len(books_df))
else:
    print('데이터를 조회하지 못했습니다.')
    
books_df

[5] 책 제목으로 출력하기
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
조건에 맞는 데이터는 총 2 건 입니다


,title,published_date,publisher,pages,recommendation
0,빅데이터 마케팅,2012.08.25,A,296,1
1,빅데이터 마케팅,2012.08.25,A,296,1


In [17]:
# find_books_by_title(db_name)
TITLE = '빅데이터'

is_success, books_df = find_books_by_title(db_name, TITLE)
if is_success:
    print('조건에 맞는 데이터는 총 %d 건 입니다.'%len(books_df))
else :
    print('데이터를 조회하지 못했습니다')

books_df

SELECT * FROM my_books WHERE title LIKE '빅데이터%' 
[4] 제목 같은 책 출력하기
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
조건에 맞는 데이터는 총 2 건 입니다.


,title,published_date,publisher,pages,recommendation
0,빅데이터 마케팅,2012.08.25,A,296,1
1,빅데이터 마케팅,2012.08.25,A,296,1


- 데이터 갱신, UPDATE

In [24]:
import sqlite3

db_name = './database/my_books.db'

### 데이터 갱신

In [25]:
import sqlite3

def update_books(db_name):
    """
    데이터를 수정하는 함수
    Args:
        db_name : Database Name
    Returns : 
        is_success : Boolean 
    """
    is_success = True
    
    try:
        conn = sqlite3.connect(db_name)
        cur = conn.cursor()
        db_sql = "UPDATE my_books SET recommendation =? WHERE title = ?"
        cur.execute(db_sql , (1, '메가트랜드'))
        
    except:
        is_success = False
        print('Database Error!!')
        
    finally:
        if is_success:
            conn.commit()
        else:
            conn.rollback()
            
        conn.close()
        
    return is_success
        

In [27]:
# select_one_book(db_name)
# update_books(db_name)
# print('[데이터 수정 완료] ================== ')
# select_one_book(db_name)

is_success, books_df1 = select_one_book(db_name)

if update_books(db_name):
    print('데이터가 성공적으로 수정되었습니다.')
else:
    print('데이터가 수정되지 않았습니다')

is_success, books_df2 = select_one_book(db_name)

books_df = pd.concat([books_df1, books_df2], axis = 0)
books_df['update'] =['수정 전', '수정 후']
books_df.set_index('update', inplace = True)
books_df

[3] 1개 데이터 출력하기
('메가트랜드', '2002.03.02', 'A', 200, 1)
데이터가 성공적으로 수정되었습니다.
[3] 1개 데이터 출력하기
('메가트랜드', '2002.03.02', 'A', 200, 1)


,title,published_date,publisher,pages,recommendation
update,,,,,
수정 전,메가트랜드,2002.03.02,A,200,1
수정 후,메가트랜드,2002.03.02,A,200,1


- 데이터 삭제, DELETE

### SQLITE3

In [29]:
import sqlite3

db_name = './database/my_books.db'

### 데이터 삭제

In [34]:
import sqlite3 

# 데이터 삭제용 함수
def delete_books_by_title(db_name, title):
    """
    책제목에 해당하는 데이터를 삭제하는 함수
    Args:
        db_name : Database Name
        title   : Title of the book to be removed
    Returns : 
        is_success : Boolean 
    """
    is_success = True
    
    try:    
        # 데이터베이스 커넥션 생성
        conn = sqlite3.connect(db_name) 

        # 커서 확보
        cur = conn.cursor()  

        # 데이터 삭제 SQL
        db_sql = "DELETE FROM my_books "
        db_sql+= "WHERE title = ?      "

        # 수정 SQL 실행
        # print('db_sql:', db_sql)
        # print('title:', title)
        cur.execute(db_sql, (title,))
        # count = cur.execute(db_sql, (title,))
        # print('count:', type(count), count)
        
    except:
        is_success = False
        print("Database Error!")
        
    finally :      
        if is_success:
            # 데이터베이스 반영
            conn.commit()  
        else:
            # 데이터베이스 철회
            conn.rollback()
            
        # 데이터베이스 커넥션 닫기
        conn.close()
    
    return is_success   

In [35]:
title = '메가트랜드'

if delete_books_by_title(db_name, title):
    print('데이터가 성공적으로 삭제되었습니다.')
    
else:
    print('데이터가 삭제되지 않았습니다')
    
is_success, books_df = select_all_books(db_name)
books_df

데이터가 성공적으로 삭제되었습니다.
[1] 전체 데이터 출력하기
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)


,title,published_date,publisher,pages,recommendation
0,인더스트리 4.0,2016.07.09,B,584,1
1,유니콘 스타트업,2011.07.15,A,248,1
2,빅데이터 마케팅,2012.08.25,A,296,1
3,사물인터넷 전망,2013.08.22,B,526,0
4,인더스트리 4.0,2016.07.09,B,584,1
5,유니콘 스타트업,2011.07.15,A,248,1
6,빅데이터 마케팅,2012.08.25,A,296,1
7,사물인터넷 전망,2013.08.22,B,526,0


In [36]:
#조건에 맞는 데이터를 삭제하는 함수

def delete_books(db_name, col_name, col_val):
    """
    조건에 맞는 데이터를 삭제하는 함수
    Args:
        db_name  : Database Name
        col_name : Column Name
        col_val  : Column Value
    Returns : 
        is_success : Boolean 
    """
    is_success = True
    
    try: 
        conn = sqlite3.connect(db_name)
        cur = conn.cursor()
        db_sql = 'DELETE FROM my_books '
        db_sql += 'WHERE {} = ?'
        db_sql = db_sql.format(col_name)
        
        cur.execute(db_sql, (col_val, ))
        
    except:
        is_success = False
        print('Database Error!')
        
    finally:
        if is_success:
            conn.commit()
            
        else:
            conn.rollback()
            
        conn.close()
        
        
    return is_success
        

In [37]:
is_success, books_df = select_all_books(db_name)
books_df

[1] 전체 데이터 출력하기
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('유니콘 스타트업', '2011.07.15', 'A', 248, 1)
('빅데이터 마케팅', '2012.08.25', 'A', 296, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)


,title,published_date,publisher,pages,recommendation
0,인더스트리 4.0,2016.07.09,B,584,1
1,유니콘 스타트업,2011.07.15,A,248,1
2,빅데이터 마케팅,2012.08.25,A,296,1
3,사물인터넷 전망,2013.08.22,B,526,0
4,인더스트리 4.0,2016.07.09,B,584,1
5,유니콘 스타트업,2011.07.15,A,248,1
6,빅데이터 마케팅,2012.08.25,A,296,1
7,사물인터넷 전망,2013.08.22,B,526,0


In [38]:
col_name = 'publisher'
col_val = 'A'
if delete_books(db_name, col_name, col_val):
    print('데이터가 성공적으로 삭제되었습니다.')
else:
    print('데이터가 삭제되지 않았습니다')
    
is_success, books_df= select_all_books(db_name)
books_df

데이터가 성공적으로 삭제되었습니다.
[1] 전체 데이터 출력하기
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('사물인터넷 전망', '2013.08.22', 'B', 526, 0)


,title,published_date,publisher,pages,recommendation
0,인더스트리 4.0,2016.07.09,B,584,1
1,사물인터넷 전망,2013.08.22,B,526,0
2,인더스트리 4.0,2016.07.09,B,584,1
3,사물인터넷 전망,2013.08.22,B,526,0


In [39]:
col_name = 'title'
col_val = '사물인터넷 전망'
if delete_books(db_name, col_name, col_val):
    print('데이터가 성공적으로 삭제되었습니다.')
else:
    print('데이터가 삭제되지 않았습니다')
    
is_success, books_df= select_all_books(db_name)
books_df

데이터가 성공적으로 삭제되었습니다.
[1] 전체 데이터 출력하기
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)
('인더스트리 4.0', '2016.07.09', 'B', 584, 1)


,title,published_date,publisher,pages,recommendation
0,인더스트리 4.0,2016.07.09,B,584,1
1,인더스트리 4.0,2016.07.09,B,584,1
